# magenta 환경 설정  
- 로컬 환경과 Colab 환경을 통해 `pip install magenta` 혹은 `curl`을 통한 자동 설치는 호환성 측면에서 많은 문제가 발생했다. (필자의 노트북은 M1 Pro)

[docker 컨테이너를 활용한 방법](https://www.notion.so/docker-6d34d8cbaf184fc3a77088349644ac49)

[Colab 환경 구성](https://www.notion.so/Colab-d804958ce7a14faeaac56b9fb39b2ac7)

In [1]:
# Install hmmlearn before downgrading Python
!pip install hmmlearn

# Downgrade Python
!apt-get update -y
!apt-get install python3.8
!update-alternatives --set python3 /usr/bin/python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py
import sys
# This path is Colab-runtime specific, check path in other systems.
_ = (sys.path.append("/usr/local/lib/python3.8/dist-packages"))


# Preinstall legacy packages
!pip install numba==0.48
!pip install numpy==1.23
!pip install packaging>=21.3
!pip install librosa==0.7.2

# Install Magenta
!pip install magenta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 11.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,011 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,773 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB

# 데이터셋 전처리 (TFRecord)

- 참고 : https://bcho.tistory.com/1190  
TFRecord 파일은 tensorflow의 학습 데이터등을 저장하기 위한 바이너리 데이터 포맷으로, 구글의 Protocol Buffer 포맷으로 데이터를 파일에 Serialize하여 저장한다.

1. CSV 파일에서와 같이 숫자나 텍스트 데이터를 읽을 때는 크게 지장이 없지만, 이미지 데이터를 읽을 경우 이미지는 JPEG나 PNG형태의 파일로 저장되어 있고 이에 대한 메타 데이터와 레이블은 별도의 파일에 저장되어 있기 때문에, 학습 데이터를 읽을 때 메타데이터나 레이블 파일 하나만 읽는 것이 아니라 이미지 파일도 별도로 읽어야 하기 때문에, 코드가 복잡해진다.

2. 이미지를 JPG나 PNG 포맷으로 읽어서 매번 디코딩을 하게되면, 그 성능이 저하되서 학습단계에서 데이터를 읽는 부분에서 많은 성능 저하가 발생한다.

In [10]:
# 데이터셋 압축 해제

import zipfile

zipfile.ZipFile('/content/groove-v1.0.0-midionly.zip').extractall('groove')

In [3]:
# info.csv (메타데이터) 확인

import pandas as pd

df = pd.read_csv('groove/groove/info.csv')
df.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [11]:
# https://github.com/magenta/magenta/tree/main/magenta/scripts
# 루트 디렉터리로부터 재귀적으로 순회하며 midi 파일 tfrecord로 변환 (내부적으로 convert_files() 함수 호출)

from magenta.scripts.convert_dir_to_note_sequences import convert_directory

convert_directory(
    root_dir='groove/',
    output_file='groove.tfrecord',
    recursive=True
)

# 모델링

In [2]:
from magenta.models.music_vae import configs

# music_vae README.md에 명시되어 있는 모델별 설정값
config_map = configs.CONFIG_MAP
config = config_map['groovae_4bar']
display(config)

# GrooVAE configs (magenta/models/music_vae/configs.py)
"""
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()),  # 4마디의 짧은 악보를 생성(?)하는 것이라 그런지.. conductor cell은 없는 것으로 추정
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,          # 배치 사이즈
            max_seq_len=16 * 4,      # 4 bars w/ 16 steps per bar
            z_size=256,              # latent dim
            enc_rnn_size=[512],      # 인코더 LSTM 셀의 hidden shape
            dec_rnn_size=[256, 256], # 디코더 LSTM 셀의 hidden shape
            max_beta=0.2,            # KL term에 대한 가중치 (1보다 작을경우 샘플링 퀄리티가 좋아짐)
            free_bits=48,
            dropout_keep_prob=0.3,   # 드롭아웃 확률
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    tfds_name='groove/4bar-midionly',
)
"""

/usr/local/lib/python3.8/dist-packages/pretty_midi/instrument.py:167: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if bend_int is not 0:
/usr/local/lib/python3.8/dist-packages/pretty_midi/instrument.py:176: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if bend_int is not 0:


Config(model=<magenta.models.music_vae.base_model.MusicVAE object at 0x7f16a3c5ffa0>, hparams=HParams([('batch_size', 512), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('control_preprocessing_rnn_size', [256]), ('dec_rnn_size', [256, 256]), ('decay_rate', 0.9999), ('dropout_keep_prob', 0.3), ('enc_rnn_size', [512]), ('free_bits', 48), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 64), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 256)]), note_sequence_augmenter=None, data_converter=<magenta.models.music_vae.data.GrooveConverter object at 0x7f16a3be00a0>, train_examples_path=None, eval_examples_path=None, tfds_name='groove/4bar-midionly')

"\nCONFIG_MAP['groovae_4bar'] = Config(\n    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),\n                   lstm_models.GrooveLstmDecoder()),\n    hparams=merge_hparams(\n        lstm_models.get_default_hparams(),\n        HParams(\n            batch_size=512,          # 배치 사이즈\n            max_seq_len=16 * 4,      # 4 bars w/ 16 steps per bar\n            z_size=256,              # latent dim\n            enc_rnn_size=[512],      # 인코더 LSTM 셀의 hidden shape\n            dec_rnn_size=[256, 256], # 디코더 LSTM 셀의 hidden shape\n            max_beta=0.2,\n            free_bits=48,\n            dropout_keep_prob=0.3,   # 드롭아웃 확률\n        )),\n    note_sequence_augmenter=None,\n    data_converter=data.GrooveConverter(\n        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,\n        max_tensors_per_notesequence=20,\n        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,\n        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),\n    tfds_name='groove/4bar-midionly',\n)

# 학습

In [6]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf

def _get_input_tensors(dataset, config):
    """Get input tensors from dataset."""
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence, sequence_length) = iterator.get_next()
    input_sequence.set_shape(
        [batch_size, None, config.data_converter.input_depth]
    )
    output_sequence.set_shape(
          [batch_size, None, config.data_converter.output_depth]
    )
    if not config.data_converter.control_depth:
        control_sequence = None
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth]
        )
    sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

    return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
    }


In [7]:
import tf_slim 

def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
    """Train loop."""
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)
        
    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(num_ps_tasks, merge_devices=True)):

            model = config.model
            model.build(
                config.hparams,
                config.data_converter.output_depth,
                is_training=True
           )

            # AdamOptimzer 사용
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,
                    num_sync_workers
                )
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)

            # gradient clip : gradient가 급격하기 explode하는 것을 방지하기 위해 임계치를 넘어가면 특정값으로 고정
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                    grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads]
                )
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode)
                )
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step'
            )

            logging_dict = {'global_step': model.global_step, 
                            'loss': model.loss}

            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                max_to_keep=checkpoints_to_keep,
                keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours)
            )
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief
            )

In [8]:
# 학습 가중치 저장 경로

import os

run_dir = os.path.expanduser('/content/music_vae/')
train_dir = os.path.join(run_dir, 'train')
train_dir

'/content/music_vae/train'

In [9]:
# 이전에 전처리한 tfrecord를 학습에 사용하도록 설정 객체 업데이트

config_update_map = {}
config_update_map['train_examples_path'] = os.path.expanduser(
    '/content/groove.tfrecord'
)
config_update_map['tfds_name'] = None
config = configs.update_config(config, config_update_map)
config

Config(model=<magenta.models.music_vae.base_model.MusicVAE object at 0x7f16a3c5ffa0>, hparams=HParams([('batch_size', 512), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('control_preprocessing_rnn_size', [256]), ('dec_rnn_size', [256, 256]), ('decay_rate', 0.9999), ('dropout_keep_prob', 0.3), ('enc_rnn_size', [512]), ('free_bits', 48), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 64), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 256)]), note_sequence_augmenter=None, data_converter=<magenta.models.music_vae.data.GrooveConverter object at 0x7f16a3be00a0>, train_examples_path='/content/groove.tfrecord', eval_examples_path=None, tfds_name=None)

In [12]:
from magenta.models.music_vae.data import get_dataset

# magenta에 내장되어 있는 데이터셋 로드 함수
# config에 명시된 train_examples_path (tfrecord)를 데이터셋화한다.

def dataset_fn():
    return get_dataset(
        config,
        tf_file_reader=tf.data.TFRecordDataset,
        is_training=True,
        cache_dataset=True
    )

In [13]:
train(
    train_dir,
    config=config,
    dataset_fn=dataset_fn,
    num_steps=1000,  # 'Number of training steps or `None` for infinite.'
)


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


# 생성

In [20]:
# Copyright 2023 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""A class for sampling, encoding, and decoding from trained MusicVAE models."""
import copy
import os
import re
import tarfile
import tempfile

import numpy as np
import tensorflow.compat.v1 as tf


class NoExtractedExamplesError(Exception):
  pass


class MultipleExtractedExamplesError(Exception):
  pass


class TrainedModel(object):
  """An interface to a trained model for encoding, decoding, and sampling.

  Attributes:
    config: The Config to build the model graph with.
    batch_size: The batch size to build the model graph with.
    checkpoint_dir_or_path: The directory containing checkpoints for the model,
      the most recent of which will be loaded, or a direct path to a specific
      checkpoint.
    var_name_substitutions: Optional list of string pairs containing regex
      patterns and substitution values for renaming model variables to match
      those in the checkpoint. Useful for backwards compatibility.
    session_target: Optional execution engine to connect to. Defaults to
      in-process.
    sample_kwargs: Additional, non-tensor keyword arguments to pass to sample
      call.
  """

  def __init__(self, config, batch_size, checkpoint_dir_or_path=None,
               var_name_substitutions=None, session_target='', **sample_kwargs):
    if tf.gfile.IsDirectory(checkpoint_dir_or_path):
      checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir_or_path)
    else:
      checkpoint_path = checkpoint_dir_or_path
    # self._config = copy.deepcopy(config)
    self._config = config  # deepcopy 오류로 인해 수정
    self._config.data_converter.set_mode('infer')
    self._config.hparams.batch_size = batch_size
    with tf.Graph().as_default():
      model = self._config.model
      model.build(
          self._config.hparams,
          self._config.data_converter.output_depth,
          is_training=False)
      # Input placeholders
      self._temperature = tf.placeholder(tf.float32, shape=())

      if self._config.hparams.z_size:
        self._z_input = tf.placeholder(
            tf.float32, shape=[batch_size, self._config.hparams.z_size])
      else:
        self._z_input = None

      if self._config.data_converter.control_depth > 0:
        self._c_input = tf.placeholder(
            tf.float32, shape=[None, self._config.data_converter.control_depth])
      else:
        self._c_input = None

      self._inputs = tf.placeholder(
          tf.float32,
          shape=[batch_size, None, self._config.data_converter.input_depth])
      self._controls = tf.placeholder(
          tf.float32,
          shape=[batch_size, None, self._config.data_converter.control_depth])
      self._inputs_length = tf.placeholder(
          tf.int32,
          shape=[batch_size] + list(self._config.data_converter.length_shape))
      self._max_length = tf.placeholder(tf.int32, shape=())
      # Outputs
      self._outputs, self._decoder_results = model.sample(
          batch_size,
          max_length=self._max_length,
          z=self._z_input,
          c_input=self._c_input,
          temperature=self._temperature,
          **sample_kwargs)
      if self._config.hparams.z_size:
        q_z = model.encode(self._inputs, self._inputs_length, self._controls)
        self._mu = q_z.loc
        self._sigma = q_z.scale.diag
        self._z = q_z.sample()

      var_map = None
      if var_name_substitutions is not None:
        var_map = {}
        for v in tf.global_variables():
          var_name = v.name[:-2]  # Strip ':0' suffix.
          for pattern, substitution in var_name_substitutions:
            var_name = re.sub(pattern, substitution, var_name)
          if var_name != v.name[:-2]:
            tf.logging.info('Renaming `%s` to `%s`.', v.name[:-2], var_name)
          var_map[var_name] = v

      # Restore graph
      self._sess = tf.Session(target=session_target)
      saver = tf.train.Saver(var_map)
      if (os.path.exists(checkpoint_path) and
          tarfile.is_tarfile(checkpoint_path)):
        tf.logging.info('Unbundling checkpoint.')
        with tempfile.TemporaryDirectory() as temp_dir:
          tar = tarfile.open(checkpoint_path)
          tar.extractall(temp_dir)
          # Assume only a single checkpoint is in the directory.
          for name in tar.getnames():
            if name.endswith('.index'):
              checkpoint_path = os.path.join(temp_dir, name[0:-6])
              break
          saver.restore(self._sess, checkpoint_path)
      else:
        saver.restore(self._sess, checkpoint_path)

  def sample(self, n=None, length=None, temperature=1.0, same_z=False,
             c_input=None):
    """Generates random samples from the model.

    Args:
      n: The number of samples to return. A full batch will be returned if not
        specified.
      length: The maximum length of a sample in decoder iterations. Required
        if end tokens are not being used.
      temperature: The softmax temperature to use (if applicable).
      same_z: Whether to use the same latent vector for all samples in the
        batch (if applicable).
      c_input: A sequence of control inputs to use for all samples (if
        applicable).
    Returns:
      A list of samples as NoteSequence objects.
    Raises:
      ValueError: If `length` is not specified and an end token is not being
        used.
    """
    batch_size = self._config.hparams.batch_size
    n = n or batch_size
    z_size = self._config.hparams.z_size

    if not length and self._config.data_converter.end_token is None:
      raise ValueError(
          'A length must be specified when the end token is not used.')
    length = length or tf.int32.max

    feed_dict = {
        self._temperature: temperature,
        self._max_length: length
    }

    if self._z_input is not None and same_z:
      z = np.random.randn(z_size).astype(np.float32)
      z = np.tile(z, (batch_size, 1))
      feed_dict[self._z_input] = z

    if self._c_input is not None:
      feed_dict[self._c_input] = c_input

    outputs = []
    for _ in range(int(np.ceil(n / batch_size))):
      if self._z_input is not None and not same_z:
        feed_dict[self._z_input] = (
            np.random.randn(batch_size, z_size).astype(np.float32))
      outputs.append(self._sess.run(self._outputs, feed_dict))
    samples = np.vstack(outputs)[:n]
    if self._c_input is not None:
      return self._config.data_converter.from_tensors(
          samples, np.tile(np.expand_dims(c_input, 0), [batch_size, 1, 1]))
    else:
      return self._config.data_converter.from_tensors(samples)

  def encode(self, note_sequences, assert_same_length=False):
    """Encodes a collection of NoteSequences into latent vectors.

    Args:
      note_sequences: A collection of NoteSequence objects to encode.
      assert_same_length: Whether to raise an AssertionError if all of the
        extracted sequences are not the same length.
    Returns:
      The encoded `z`, `mu`, and `sigma` values.
    Raises:
      RuntimeError: If called for a non-conditional model.
      NoExtractedExamplesError: If no examples were extracted.
      MultipleExtractedExamplesError: If multiple examples were extracted.
      AssertionError: If `assert_same_length` is True and any extracted
        sequences differ in length.
    """
    if not self._config.hparams.z_size:
      raise RuntimeError('Cannot encode with a non-conditional model.')

    inputs = []
    controls = []
    lengths = []
    for note_sequence in note_sequences:
      extracted_tensors = self._config.data_converter.to_tensors(note_sequence)
      if not extracted_tensors.inputs:
        raise NoExtractedExamplesError(
            'No examples extracted from NoteSequence: %s' % note_sequence)
      if len(extracted_tensors.inputs) > 1:
        raise MultipleExtractedExamplesError(
            'Multiple (%d) examples extracted from NoteSequence: %s' %
            (len(extracted_tensors.inputs), note_sequence))
      inputs.append(extracted_tensors.inputs[0])
      controls.append(extracted_tensors.controls[0])
      lengths.append(extracted_tensors.lengths[0])
      if assert_same_length and len(inputs[0]) != len(inputs[-1]):
        raise AssertionError(
            'Sequences 0 and %d have different lengths: %d vs %d' %
            (len(inputs) - 1, len(inputs[0]), len(inputs[-1])))
    return self.encode_tensors(inputs, lengths, controls)

  def encode_tensors(self, input_tensors, lengths, control_tensors=None):
    """Encodes a collection of input tensors into latent vectors.

    Args:
      input_tensors: Collection of input tensors to encode.
      lengths: Collection of lengths of input tensors.
      control_tensors: Collection of control tensors to encode.
    Returns:
      The encoded `z`, `mu`, and `sigma` values.
    Raises:
       RuntimeError: If called for a non-conditional model.
    """
    if not self._config.hparams.z_size:
      raise RuntimeError('Cannot encode with a non-conditional model.')

    n = len(input_tensors)
    input_depth = self._config.data_converter.input_depth
    batch_size = self._config.hparams.batch_size

    batch_pad_amt = -n % batch_size
    if batch_pad_amt > 0:
      input_tensors += [np.zeros([0, input_depth])] * batch_pad_amt
    length_array = np.array(lengths, np.int32)
    length_array = np.pad(
        length_array,
        [(0, batch_pad_amt)] + [(0, 0)] * (length_array.ndim - 1),
        'constant')

    max_length = max([len(t) for t in input_tensors])
    inputs_array = np.zeros(
        [len(input_tensors), max_length, input_depth])
    for i, t in enumerate(input_tensors):
      inputs_array[i, :len(t)] = t

    control_depth = self._config.data_converter.control_depth
    controls_array = np.zeros(
        [len(input_tensors), max_length, control_depth])
    if control_tensors is not None:
      control_tensors += [np.zeros([0, control_depth])] * batch_pad_amt
      for i, t in enumerate(control_tensors):
        controls_array[i, :len(t)] = t

    outputs = []
    for i in range(len(inputs_array) // batch_size):
      batch_begin = i * batch_size
      batch_end = (i+1) * batch_size
      feed_dict = {self._inputs: inputs_array[batch_begin:batch_end],
                   self._controls: controls_array[batch_begin:batch_end],
                   self._inputs_length: length_array[batch_begin:batch_end]}
      outputs.append(
          self._sess.run([self._z, self._mu, self._sigma], feed_dict))
    assert outputs
    return tuple(np.vstack(v)[:n] for v in zip(*outputs))

  def decode(self, z, length=None, temperature=1.0, c_input=None):
    """Decodes a collection of latent vectors into NoteSequences.

    Args:
      z: A collection of latent vectors to decode.
      length: The maximum length of a sample in decoder iterations. Required
        if end tokens are not being used.
      temperature: The softmax temperature to use (if applicable).
      c_input: Control sequence (if applicable).
    Returns:
      A list of decodings as NoteSequence objects.
    Raises:
      RuntimeError: If called for a non-conditional model.
      ValueError: If `length` is not specified and an end token is not being
        used.
    """
    tensors = self.decode_to_tensors(z, length, temperature, c_input)
    if self._c_input is not None:
      return self._config.data_converter.from_tensors(
          tensors,
          np.tile(
              np.expand_dims(c_input, 0),
              [self._config.hparams.batch_size, 1, 1]))
    else:
      return self._config.data_converter.from_tensors(tensors)

  def decode_to_tensors(self, z, length=None, temperature=1.0, c_input=None,
                        return_full_results=False):
    """Decodes a collection of latent vectors into output tensors.

    Args:
      z: A collection of latent vectors to decode.
      length: The maximum length of a sample in decoder iterations. Required
        if end tokens are not being used.
      temperature: The softmax temperature to use (if applicable).
      c_input: Control sequence (if applicable).
      return_full_results: If true will return the full decoder_results,
        otherwise it will return only the samples.
    Returns:
      If return_full_results is True, will return the full decoder_results list,
      otherwise it will return the samples from the decoder as a 2D numpy array.
    Raises:
      RuntimeError: If called for a non-conditional model.
      ValueError: If `length` is not specified and an end token is not being
        used.
    """
    if not self._config.hparams.z_size:
      raise RuntimeError('Cannot decode with a non-conditional model.')

    if not length and self._config.data_converter.end_token is None:
      raise ValueError(
          'A length must be specified when the end token is not used.')
    batch_size = self._config.hparams.batch_size
    n = len(z)
    length = length or tf.int32.max

    batch_pad_amt = -n % batch_size
    z = np.pad(z, [(0, batch_pad_amt), (0, 0)], mode='constant')

    outputs = []
    for i in range(len(z) // batch_size):
      feed_dict = {
          self._temperature: temperature,
          self._z_input: z[i*batch_size:(i+1)*batch_size],
          self._max_length: length,
      }
      if self._c_input is not None:
        feed_dict[self._c_input] = c_input
      if return_full_results:
        outputs.extend(self._sess.run(self._decoder_results, feed_dict))
      else:
        outputs.extend(self._sess.run(self._outputs, feed_dict))
    return outputs[:n]

  def interpolate(self, start_sequence, end_sequence, num_steps,
                  length=None, temperature=1.0, assert_same_length=True):
    """Interpolates between a start and an end NoteSequence.

    Args:
      start_sequence: The NoteSequence to interpolate from.
      end_sequence: The NoteSequence to interpolate to.
      num_steps: Number of NoteSequences to be generated, including the
        reconstructions of the start and end sequences.
      length: The maximum length of a sample in decoder iterations. Required
        if end tokens are not being used.
      temperature: The softmax temperature to use (if applicable).
      assert_same_length: Whether to raise an AssertionError if all of the
        extracted sequences are not the same length.
    Returns:
      A list of interpolated NoteSequences.
    Raises:
      AssertionError: If `assert_same_length` is True and any extracted
        sequences differ in length.
    """
    def _slerp(p0, p1, t):
      """Spherical linear interpolation."""
      omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)),
                               np.squeeze(p1/np.linalg.norm(p1))))
      so = np.sin(omega)
      return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

    _, mu, _ = self.encode([start_sequence, end_sequence], assert_same_length)
    z = np.array([_slerp(mu[0], mu[1], t)
                  for t in np.linspace(0, 1, num_steps)])
    return self.decode(
        length=length,
        z=z,
        temperature=temperature)

In [26]:
# Copyright 2023 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""MusicVAE generation script."""

# TODO(adarob): Add support for models with conditioning.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

from magenta.models.music_vae import configs
# from magenta.models.music_vae import TrainedModel
import note_seq
import numpy as np
import tensorflow.compat.v1 as tf


logging = tf.logging
run_dir = '/content/music_vae'
output_dir = '/content/music_vae/generated'
num_outputs = 5
max_batch_size = 8
temperature = 0.5


def run(config_map):
    """Load model params, save config file and start trainer.

    Args:
      config_map: Dictionary mapping configuration name to Config object.

    Raises:
      ValueError: if required flags are missing or invalid.
    """
    config = config_map['groovae_4bar']
    date_and_time = time.strftime('%Y-%m-%d_%H%M%S')
    config.data_converter.max_tensors_per_item = None

    logging.info('Loading model...')
    checkpoint_dir_or_path = os.path.expanduser(
        os.path.join(run_dir, 'train')
    )
    # pre-trained model weight
    # checkpoint_dir_or_path = os.path.expanduser(
    #     'groovae_4bar.tar'
    # )
    # 학습된 모델 가중치 체크포인트(.ckpt)가 저장된 경로를 인자로 넘겨 모델 로드
    model = TrainedModel(
      config, 
      batch_size=min(max_batch_size, num_outputs),
      checkpoint_dir_or_path=checkpoint_dir_or_path
    )

    # 모델을 통해 확률 분포 공간으로부터 노트 샘플링
    logging.info('Sampling...')
    results = model.sample(
        n=num_outputs,
        length=config.hparams.max_seq_len,
        temperature=temperature)

    basename = os.path.join(
      output_dir,
      '%s_%s-*-of-%03d.mid' %
      ('sample', date_and_time, num_outputs))
    logging.info('Outputting %d files as `%s`...', num_outputs, basename)
    # 노트 시퀀스 미디 파일(.mid)로 변환
    for i, ns in enumerate(results):
        note_seq.sequence_proto_to_midi_file(ns, basename.replace('*', '%03d' % i))

    logging.info('Done.')


In [27]:
run(config_map)

In [29]:
!tar -zcvf music_vae.tar.gz /content/music_vae/

tar: Removing leading `/' from member names
/content/music_vae/
/content/music_vae/generated/
/content/music_vae/generated/sample_2023-06-02_140601-002-of-005.mid
/content/music_vae/generated/sample_2023-06-02_140601-003-of-005.mid
/content/music_vae/generated/sample_2023-06-02_140601-001-of-005.mid
/content/music_vae/generated/sample_2023-06-02_140601-000-of-005.mid
/content/music_vae/generated/sample_2023-06-02_140601-004-of-005.mid
/content/music_vae/.ipynb_checkpoints/
/content/music_vae/train/
/content/music_vae/train/model.ckpt-758.meta
/content/music_vae/train/model.ckpt-1000.meta
/content/music_vae/train/model.ckpt-758.data-00000-of-00001
/content/music_vae/train/model.ckpt-483.index
/content/music_vae/train/model.ckpt-758.index
/content/music_vae/train/model.ckpt-899.data-00000-of-00001
/content/music_vae/train/model.ckpt-899.index
/content/music_vae/train/model.ckpt-1000.index
/content/music_vae/train/.ipynb_checkpoints/
/content/music_vae/train/model.ckpt-1000.data-00000-of-